# Hybrid Quantum-Classical Medical Imaging on Colab

This notebook sets up the environment to run the QML medical imaging project directly from GitHub using Google Colab.

## key Features
- Clones the repository from GitHub (Supports Private Repos).
- Installs dependencies.
- Mounts Google Drive for persistent data and output storage.
- Runs training/inference scripts.

In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define paths
DATASET_PATH = '/content/drive/MyDrive/medical_imaging_data/brain_tumor_dataset' # @param {type:"string"}
OUTPUT_PATH = '/content/drive/MyDrive/qml_medical_imaging_outputs' # @param {type:"string"}

In [ ]:
# 2. Clone Repository
import os
import shutil
from getpass import getpass

user = "DreamX55"
repo = "QML-for-Medical-Imaging"

if not os.path.exists(repo):
    print("Is your repository PRIVATE? If yes, you need a Personal Access Token (PAT).")
    print("Get one here: https://github.com/settings/tokens (Select 'repo' scope)")
    token = getpass('Enter GitHub PAT (leave empty if public): ')

    if token:
        !git clone https://{token}@github.com/{user}/{repo}.git
    else:
        !git clone https://github.com/{user}/{repo}.git

%cd {repo}
!pip install -r requirements.txt

In [ ]:
# 3. Symlink Data and Outputs
# This allows the code to run without changing paths, while storing heavy files in Drive
import os
import shutil

def force_symlink(src, dst):
    if os.path.exists(dst) or os.path.islink(dst):
        if os.path.isdir(dst) and not os.path.islink(dst):
            shutil.rmtree(dst)
        else:
            os.remove(dst)
    os.symlink(src, dst)
    print(f"Linked {src} -> {dst}")

# Ensure parent directories exist
os.makedirs('data', exist_ok=True)

# Create links
force_symlink(DATASET_PATH, 'data/brain_mri')
force_symlink(OUTPUT_PATH, 'outputs')

In [ ]:
# 4. Run Training
!python main.py --config configs/hybrid_model.yaml